In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [9]:
compras = (pd.concat
                ([
                pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'),
                productos_sin_compra
               ])
        )


cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'] - pd.Timedelta(days=1)

In [10]:
costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

# Checks

In [11]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Todo correcto con check1


In [12]:
check2 = costo_venta[costo_venta['product_price'].isna()][['product_id', 'product_name_x']]
print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 554 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
13,11641,"Pija Drywall 8 x 1 1/2"" 15421 (Kg) *16293*"
43,12847,Chapa Dexter #2021 X-300 Gancho ADK Alum *17821*
55,11173,"Pija Drywall 8 x 2 1/2"" 15423 (Kg) *16290*"
77,8436,Jal. Pte. 64/124mm AI Negro HH #79997 *79997*
397,10707,Catal Sayer KC-0511 (95Gr) Rojo *12904*
...,...,...
58201,29541,USGMx Redimix mini cubeta (6kg) *5175*
58273,29540,Escalera Tijera ESCG70806 *5170*
58747,29544,Picaporte P/Porton 40 Cm Lock LP40 *5193*
59990,29553,"Caja Porta Herramienta 21"" CPS20 *5235*"


# Pruebas

In [ ]:
product_id_to_check = 13811

costo_venta.loc[costo_venta['product_id'] == product_id_to_check][['name', 'invoice_date', 'salesperson_name', 'quantity', 'price_subtotal', 'product_price', 'order_id', 'order_name',]].to_excel('a.xlsx')

In [21]:
compras.loc[compras['product_id_pp'] == product_id_to_check].groupby(['product_price']).first()
compras.loc[(compras['product_id_pp'] == product_id_to_check) & (compras['order_id'] > 1)]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
1,19070,3117,P03103 (MPIFCO 12908),2024-06-25 22:19:17,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,5.00,469.83,False,MPIFCO 12908,2024-06-25,Alexa Yadira Mazariegos Zunun,Alexa Yadira Mazariegos Zunun,2024-06-24 22:19:17
925,17447,2886,P02872 (B 86851),2024-06-11 23:43:35,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,5.00,514.55,False,B 86851,2024-06-11,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-10 23:43:35
1391,16584,2779,P02766 (SJCFCO 14666),2024-06-05 23:56:24,5326,Madereria El Pino De Los Cabos,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,80.00,443.97,False,SJCFCO 14666,2024-06-05,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez,2024-06-04 23:56:24
1633,16164,2718,P02705 (B 86651),2024-06-04 00:19:47,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,100.00,443.34,False,B 86651,2024-06-03,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores,2024-06-03 00:19:47
1635,16160,2717,P02704 (MPIFCO 11393),2024-06-04 00:17:31,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,10.00,456.90,False,MPIFCO 11393,2024-06-03,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-03 00:17:31
2081,15405,2633,P02620 (MPIFCO 11004),2024-05-29 22:53:25,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,6.00,456.90,False,MPIFCO 11004,2024-05-29,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores,2024-05-28 22:53:25
2115,15339,2628,P02615 (B 86475),2024-05-28 23:56:28,5290,Maria Reyna Navarrete Cortez,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,1.00,443.34,False,B 86475,2024-05-28,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-05-27 23:56:28
2176,15244,2598,P02585 (SJCFCO 13661),2024-05-25 15:41:29,5326,Madereria El Pino De Los Cabos,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,3.00,456.90,False,SJCFCO 13661,2024-05-25,Gretel Gómez Alcázar,Yamilet Blanco Salas,2024-05-24 15:41:29
2352,14910,2583,P02570 (A 67401),2024-05-24 20:59:16,5343,Irma Rodriguez Montoya,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,3.00,469.83,False,A 67401,2024-05-24,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez,2024-05-23 20:59:16
2501,14642,2545,P02532 (SJCFCO 13438),2024-05-24 16:04:05,5326,Madereria El Pino De Los Cabos,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,1.00,456.90,False,SJCFCO 13438,2024-05-22,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-05-23 16:04:05


In [15]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [16]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]

In [17]:
prod_varios_prov = compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']].groupby('product_id_pp').agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
prod_varios_prov.columns = ['product_name', 'mean', 'first']
prod_varios_prov['diff'] = prod_varios_prov['mean']  == prod_varios_prov['first']
prod_varios_prov.loc[prod_varios_prov['diff'] == False]

,product_name,mean,first,diff
product_id_pp,,,,
7480,"[2538] Rondana Plana 1/4"" C#181 *2538*","5,337.00",5366,False
7481,[2540] Rondana Plana 3/16 C#184 *2540*,"5,293.50",5366,False
7483,"[6528] Pija Autorr. C/Rondana 8""*1/2"" Galv. *6...","5,345.29",5366,False
7510,"[5388] Tuerca Hexagonal 1/4"" Hncz (5600/Ct) C...","5,329.75",5366,False
7511,"[10179] Taq. Plastico El-Pro 1/4"" Blanco TQ-01...","5,328.20",5313,False
...,...,...,...,...
29236,[4413] Brida Flexible Coflex PB-300 Larga *4413*,"5,326.33",5335,False
29241,[4426] Color Cemento Negro *4426*,"5,236.50",5143,False
29255,[4497] Guantes De Corte De Nitrilo 48-22-8902 ...,"5,179.80",5128,False


In [18]:
prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp']

0       7480
1       7481
2       7483
3       7510
4       7511
       ...  
205    29236
206    29241
207    29255
208    29436
209    29452
Name: product_id_pp, Length: 210, dtype: Int64

In [19]:
compras[compras['product_id_pp'].isin(prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp'])]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
1,19070,3117,P03103 (MPIFCO 12908),2024-06-25 22:19:17,5296,Madereria Pichonchas,13811,[3651] Triplay Pino 1/2 4'*8' 1/Cara *3651*,5.00,469.83,False,MPIFCO 12908,2024-06-25,Alexa Yadira Mazariegos Zunun,Alexa Yadira Mazariegos Zunun,2024-06-24 22:19:17
2,19071,3117,P03103 (MPIFCO 12908),2024-06-25 22:19:17,5296,Madereria Pichonchas,13075,[13555] Polin 4 X 4 X 8 *13555*,4.00,250.00,False,MPIFCO 12908,2024-06-25,Alexa Yadira Mazariegos Zunun,Alexa Yadira Mazariegos Zunun,2024-06-24 22:19:17
3,19072,3117,P03103 (MPIFCO 12908),2024-06-25 22:19:17,5296,Madereria Pichonchas,14156,[15293] Triplay Birch 3/4 4'*8' *15293*,2.00,724.14,False,MPIFCO 12908,2024-06-25,Alexa Yadira Mazariegos Zunun,Alexa Yadira Mazariegos Zunun,2024-06-24 22:19:17
7,19065,3114,P03100 (MPIFCO 12981),2024-06-25 22:00:42,5296,Madereria Pichonchas,26426,[96847] Tablon Douglas Fire #2 2x12x16 *96847*,2.00,"1,241.38",False,MPIFCO 12981,2024-06-25,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez,2024-06-24 22:00:42
9,19063,3112,P03098 (LC 9729),2024-06-25 19:03:19,5181,Vector De Baja California,28239,[11780] Tablaroca 1/2 1.22*2.44 *11780*,10.00,240.00,False,LC 9729,2024-06-25,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-24 19:03:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,<NA>,<NA>,SAE,2023-10-16 00:00:00,<NA>,NaN,14016,Maletin D/Broca SDS-Plus MK D-31734 13pz *8635*,NaN,599.00,NaN,NaN,NaT,NaN,NaN,2023-10-15 00:00:00
6085,<NA>,<NA>,SAE,2023-11-23 00:00:00,<NA>,NaN,14671,Malla Sombra 2x100m 90% Beige Raschel *96679*,NaN,"3,620.70",NaN,NaN,NaT,NaN,NaN,2023-11-22 00:00:00
6768,<NA>,<NA>,SAE,2023-01-19 00:00:00,<NA>,NaN,27422,Silicon Cafe DAP Alex Plus 300 ml *2468*,NaN,82.76,NaN,NaN,NaT,NaN,NaN,2023-01-18 00:00:00
7262,<NA>,<NA>,SAE,2023-11-08 00:00:00,<NA>,NaN,28253,Detector Tipo Lapiz 1AC-A1-II Mca Fluke *11508*,NaN,828.25,NaN,NaN,NaT,NaN,NaN,2023-11-07 00:00:00
